## General imports

The `inverted_index_colab` import requires the `inverted_index_colab.py` file. You should upload the file and then run this cell.

In [1]:
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import json
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

openjdk-8-jdk-headless is already the newest version (8u352-ga-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
--2023-01-04 15:03:51--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 18.154.110.122, 18.154.110.115, 18.154.110.33, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|18.154.110.122|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ not modified on server. Omitting download.



In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [5]:
spark

In [6]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth


def handler(signum, frame):
  raise Exception("Authentication timeout!")

try:
   auth.authenticate_user()
except: 
   pass

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ofek_alon_project'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.startswith('multi'):
        paths.append(full_path+b.name)

In [8]:
ids = []
with open('/content/queries_train.json') as json_file:
    data = json.load(json_file)
    for value in data.values():
      ids += value
ids = list(set(ids))
len(ids)

1189

In [9]:
# # Copy one wikidumps files 
# import os
# from pathlib import Path
# ## RENAME the project_id to yours project id from the project you created in GCP 
# project_id = 'assignment3-370316'
# !gcloud config set project {project_id}

# data_bucket_name = 'ofek_alon_project'
# try:
#     if os.environ["wikidata_preprocessed"] is not None:
#         pass  
# except:
#       !mkdir wikidumps
#       !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


In [10]:
# from pathlib import Path 
# import os

# try:
#     if os.environ["wikidata_preprocessed"] is not None:
#       path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
# except:
#       path = "wikidumps/*"

In [11]:
from preprocess_colab import PreProcess
from tokenizer import Tokenizer
from pickle_handler import PickleHandler
pickle_handler = PickleHandler()

In [12]:
indices = [('text', False), ('text', True), ('title', False), ('anchor_text', False)]
# indices = [('text', False)]

In [13]:
preprocess = PreProcess(spark)
for index in indices:
  index_type, stemming = index
  if stemming:
        index_name = index_type + "_with_stemming"
  else:
        index_name = index_type
  base_dir = './' + index_name
  base_path = Path(base_dir)
  if not base_path.exists():
    os.mkdir(index_name)
  anchor = index_type == 'anchor_text'
  postings, num_of_docs = preprocess.load_and_process(path=path, index_type=index_type, stemming=stemming, anchor=anchor)
  posting_locs_list = preprocess.partition_postings_and_write(postings, base_dir).collect()
  super_posting_locs = preprocess.get_super_posting_locs(posting_locs_list)
  inverted_index = InvertedIndex()
  inverted_index.posting_locs = super_posting_locs
  inverted_index.df = preprocess.calculate_df(postings)
  if index_type == 'text':
    inverted_index.idf = preprocess.calculate_idf(postings, num_of_docs)
    inverted_index.term_total = preprocess.calculate_term_total(postings)
    tf_idf_dict = preprocess.calculate_tf_idf(postings, inverted_index.idf)
    pickle_handler.write_pickle_file(f'tf_idf_{index_name}.pkl', tf_idf_dict)
  inverted_index.write_index(index_name, 'index')

Py4JJavaError: ignored

In [12]:
i = 0
for k, v in inverted_index.items():
  print(f'{k}: {v}')
  i += 1
  if i == 5:
    break

anarchism: [(12, 690.1826147327807), (339, 33.128765507173476), (983, 5.521460917862246), (1023, 430.6739515932552)]
political: [(12, 32.18473325604205), (303, 6.189371780008086), (307, 29.708984544038813), (308, 12.378743560016172), (336, 1.2378743560016172), (339, 24.757487120032344), (358, 18.568115340024256), (359, 2.4757487120032344), (569, 24.757487120032344), (594, 1.2378743560016172), (600, 9.902994848012938), (620, 13.61661791601779), (624, 6.189371780008086), (627, 3.7136230680048516), (643, 2.4757487120032344), (663, 2.4757487120032344), (665, 2.4757487120032344), (676, 1.2378743560016172), (677, 1.2378743560016172), (689, 4.951497424006469), (690, 2.4757487120032344), (691, 11.140869204014555), (700, 7.427246136009703), (701, 27.23323583203558), (705, 3.7136230680048516), (706, 1.2378743560016172), (717, 4.951497424006469), (736, 3.7136230680048516), (737, 11.140869204014555), (738, 11.140869204014555), (740, 1.2378743560016172), (746, 12.378743560016172), (751, 1.237874356

In [15]:
query = 'movement philosophy'
docs_to_tf_idf = defaultdict(dict)
for word in query.split(' '):
  docs = inverted_index.tf_idf[word]
  for doc in docs:
    doc_id = doc[0]
    docs_to_tf_idf[doc_id].update({word: doc[1]})

  # docs_to_tf_idf.update(
  #     {doc_id: {word: tf_idf} for (doc_id, tf_idf) in docs}
print(docs_to_tf_idf)

defaultdict(<class 'dict'>, {12: {'movement': 33.702764074587904, 'philosophy': 12.014883003262245}, 25: {'movement': 5.861350273841374}, 303: {'movement': 7.326687842301718}, 307: {'movement': 4.396012705381031, 'philosophy': 2.0024805005437076}, 308: {'movement': 7.326687842301718, 'philosophy': 56.06945401522381}, 336: {'movement': 4.396012705381031, 'philosophy': 14.017363503805953}, 339: {'movement': 11.722700547682749, 'philosophy': 60.07441501631123}, 358: {'movement': 1.4653375684603436}, 569: {'movement': 1.4653375684603436, 'philosophy': 8.00992200217483}, 593: {'movement': 17.584050821524123}, 600: {'movement': 1.4653375684603436}, 620: {'movement': 2.930675136920687, 'philosophy': 4.004961001087415}, 621: {'movement': 2.930675136920687}, 624: {'movement': 2.930675136920687}, 627: {'movement': 2.930675136920687, 'philosophy': 2.0024805005437076}, 628: {'movement': 2.930675136920687, 'philosophy': 10.012402502718539}, 653: {'movement': 4.396012705381031}, 655: {'movement': 1.

In [29]:
df = pd.DataFrame(docs_to_tf_idf).transpose().fillna(0)
cols = df.columns
index = df.index
df

,movement,philosophy
12,33.702764,12.014883
25,5.861350,0.000000
303,7.326688,0.000000
307,4.396013,2.002481
308,7.326688,56.069454
...,...,...
2349,0.000000,2.002481
2371,0.000000,2.002481
2422,0.000000,4.004961
2425,0.000000,2.002481


In [30]:
from sklearn.preprocessing import normalize
pd.DataFrame(normalize(df, axis=0), columns=cols, index=index)

,movement,philosophy
12,0.351894,0.049096
25,0.061199,0.000000
303,0.076499,0.000000
307,0.045899,0.008183
308,0.076499,0.229116
...,...,...
2349,0.000000,0.008183
2371,0.000000,0.008183
2422,0.000000,0.016365
2425,0.000000,0.008183


In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
pickle_handler.write_pickle_file(pv_clean, wid2pv)
# with open(pv_clean, 'wb') as f:
#   pickle.dump(wid2pv, f)



--2023-01-03 17:34:12--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2503235912 (2.3G) [application/octet-stream]
Saving to: ‘pageviews-202108-user.bz2’

pageviews-202108-us 100%[===================>]   2.33G  4.62MB/s    in 9m 12s  

2023-01-03 17:43:23 (4.33 MB/s) - ‘pageviews-202108-user.bz2’ saved [2503235912/2503235912]



In [ ]:
# read in the counter
# with open(pv_clean, 'rb') as f:
#   wid2pv = pickle.loads(f.read())

pickle_handler.read_pickle_file(pv_clean)

NameError: ignored

In [ ]:
doc_title_pairs, _ = preprocess.read_files(path, 'title')
title_id_dict = {row.id: row.title for row in doc_title_pairs.collect()}
pickle_handler.write_pickle_file('title.pkl', title_id_dict)

In [ ]:
d = pickle_handler.read_pickle_file('title.pkl')
d

{12: 'Anarchism',
 25: 'Autism',
 39: 'Albedo',
 290: 'A',
 303: 'Alabama',
 305: 'Achilles',
 307: 'Abraham Lincoln',
 308: 'Aristotle',
 309: 'An American in Paris',
 316: 'Academy Award for Best Production Design',
 324: 'Academy Awards',
 330: 'Actrius',
 332: 'Animalia (book)',
 334: 'International Atomic Time',
 336: 'Altruism',
 339: 'Ayn Rand',
 340: 'Alain Connes',
 344: 'Allan Dwan',
 358: 'Algeria',
 359: 'List of Atlas Shrugged characters',
 569: 'Anthropology',
 572: 'Agricultural science',
 573: 'Alchemy',
 579: 'Alien',
 580: 'Astronomer',
 586: 'ASCII',
 590: 'Austin (disambiguation)',
 593: 'Animation',
 594: 'Apollo',
 595: 'Andre Agassi',
 597: 'Austroasiatic languages',
 599: 'Afroasiatic languages',
 600: 'Andorra',
 612: 'Arithmetic mean',
 615: 'American Football Conference',
 620: 'Animal Farm',
 621: 'Amphibian',
 624: 'Alaska',
 627: 'Agriculture',
 628: 'Aldous Huxley',
 630: 'Ada',
 632: 'Aberdeen (disambiguation)',
 633: 'Algae',
 634: 'Analysis of variance